In [1]:
#Import Python Libraries
import requests
import pandas as pd
import sys
from io import StringIO
from datetime import datetime
from time import gmtime, strftime

#API GET Request
url = "https://banks.data.fdic.gov/api/failures?filters=%21%28STATE%3A%22NE%22%2C%22KS%22%2C%22FL%22%29&fields=NAME%2CCITYST%2CFAILDATE&limit=5000&offset=0&agg_limit=10&format=csv&download=false&filename=data_file"
headers = {'accept': 'application/json'}

#The API GET request returns the following data in CSV format as a string.
# 1. Data should not contain bank from state NE,KS and FL
# 2. Data only contain bank name, city, state, closing date and current timestamp

#StringIO() library used to convert csv string to file format.
stdout = StringIO(requests.get(url, headers=headers).text)

#The stdout is converted to a dataframe using Pandas library.
df = pd.read_csv(stdout, index_col = False)

#Split CITYST into CITY and STATE
df[['CITY','STATE']] = df.CITYST.str.split(',',1,expand=True)

#Return only required columns.
df = df[['NAME','CITY','STATE','FAILDATE']]

#Add dynamic timestamp
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S") + strftime("%z", gmtime())
df['current_tmpstmp'] = current_time

#Change Date Format to follow PDF
df['FAILDATE']=pd.to_datetime(df['FAILDATE'], format='%m/%d/%Y').dt.strftime('%b %d, %Y')
df['current_tmpstmp']=pd.to_datetime(df['current_tmpstmp'], format='%Y-%m-%d %H:%M:%S%z')


#Sort by State Asc and City Desc
df = df.sort_values(['STATE', 'CITY'], ascending=[True,False])

#Rename Columns
df.rename(columns = {'NAME':'bank_nm', 'CITY':'city', 'STATE':'state' , 'FAILDATE':'close_dt'}, inplace = True)

df

,bank_nm,city,state,close_dt,current_tmpstmp
250,PENINSULA FS&LA,SOLDOTNA,AK,"Aug 08, 1986",2021-05-26 01:51:25+08:00
456,FRONTIER S&LA,FAIRBANKS,AK,"Jun 12, 1987",2021-05-26 01:51:25+08:00
547,ALASKA NATIONAL BANK OF THE NORTH,FAIRBANKS,AK,"Oct 22, 1987",2021-05-26 01:51:25+08:00
3351,ARCTIC FS&LA,FAIRBANKS,AK,"May 09, 1981",2021-05-26 01:51:25+08:00
288,SECURITY NATIONAL BANK,ANCHORAGE,AK,"Oct 23, 1986",2021-05-26 01:51:25+08:00
...,...,...,...,...,...
618,BANK OF CASPER,CASPER,WY,"Jan 22, 1988",2021-05-26 01:51:25+08:00
1728,PROVIDENT SA,CASPER,WY,"Feb 23, 1990",2021-05-26 01:51:25+08:00
3639,WESTERN NATIONAL BK OF CASPER,CASPER,WY,"May 04, 1984",2021-05-26 01:51:25+08:00
285,VALLEY STATE BANK,BAGGS,WY,"Oct 17, 1986",2021-05-26 01:51:25+08:00


In [4]:
#Import Libraries
import pdfkit as pdf

#Windows64 version requires installation of wkhtmltopdf for conversion to work. Also requires to specify the path and config.
path_wkhtmltopdf = r'C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe'
config = pdf.configuration(wkhtmltopdf=path_wkhtmltopdf)

#Pandas dataframe to html file
df.to_html('C:\\Users\\SUNBTC\\Desktop\\Air_Asia_Interview\\Assessment_3\Output\\Failed_Banks.html', index=False)

#Conversion from html to PDF
Failed_Banks='C:\\Users\\SUNBTC\\Desktop\\Air_Asia_Interview\\Assessment_3\Output\\Failed_Banks.pdf'
pdf.from_file('C:\\Users\\SUNBTC\\Desktop\\Air_Asia_Interview\\Assessment_3\\Output\\Failed_Banks.html', Failed_Banks, configuration = config)




Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


True